# Battle of the Neighborhoods

## Introduction

New York City, now the epicenter of the novel coronavirus pandemic, is looking for locations to set up locations to take in patients with symptoms of COVID-19. 

In this project, we will try to find new hospital locations in all of the five boroughs: 
  * Bronx (9,936 cases as of 4/3), 
  * Brooklyn (14,420), 
  * Manhattan(7,713), 
  * Queens (17,832), and 
  * Staten Island (3,012).  



Factors to consider:
  * Distance from existing hospital
  * Population  
  * Population density
  * Hospital beds per person

## Data

The Foursquare API will be used to locate existing hospitals to identify new areas where additional medical facilities could be set up to help mitigate the inflow of new patients arriving at existing hospitals. Data from the NYC Open Data website will also be used for geospatial and population data. Data will have:
  * neighborhood names, 
  * borough names, 
  * population, 
  * latitude and longitude coordinates of neighborhoods and 
  * latitude and longitude coordinates of existing hospitals.

### Import libraries.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Get New York City's geospatial data.

In [2]:
geo_url = 'https://data.cityofnewyork.us/api/views/xyye-rtrs/rows.csv?accessType=DOWNLOAD'
df2 = pd.read_csv(geo_url)
print(df2.shape)
df2.head()

(299, 9)


,the_geom,OBJECTID,Name,Stacked,AnnoLine1,AnnoLine2,AnnoLine3,AnnoAngle,Borough
0,POINT (-73.8472005205491 40.89470517661004),1,Wakefield,1,Wakefield,NaN,NaN,0,Bronx
1,POINT (-73.82993910812405 40.87429419303015),2,Co-op City,2,Co-op,City,NaN,0,Bronx
2,POINT (-73.82780644716419 40.88755567735082),3,Eastchester,1,Eastchester,NaN,NaN,0,Bronx
3,POINT (-73.90564259591689 40.895437426903875),4,Fieldston,1,Fieldston,NaN,NaN,0,Bronx
4,POINT (-73.91258546108577 40.89083449389134),5,Riverdale,1,Riverdale,NaN,NaN,0,Bronx


### Clean and format the dataframe.

In [3]:
df2['the_geom'] = df2['the_geom'].map(lambda x: x.lstrip('POINT (').rstrip(')'))

df2.head()

,the_geom,OBJECTID,Name,Stacked,AnnoLine1,AnnoLine2,AnnoLine3,AnnoAngle,Borough
0,-73.8472005205491 40.89470517661004,1,Wakefield,1,Wakefield,NaN,NaN,0,Bronx
1,-73.82993910812405 40.87429419303015,2,Co-op City,2,Co-op,City,NaN,0,Bronx
2,-73.82780644716419 40.88755567735082,3,Eastchester,1,Eastchester,NaN,NaN,0,Bronx
3,-73.90564259591689 40.895437426903875,4,Fieldston,1,Fieldston,NaN,NaN,0,Bronx
4,-73.91258546108577 40.89083449389134,5,Riverdale,1,Riverdale,NaN,NaN,0,Bronx


In [4]:
new = df2['the_geom'].str.split(" ", n = 1, expand = True)

df2['long']= new[0]

df2['lat'] = new[1]

df2.drop(columns =['the_geom'], inplace = True)

df2.head()

,OBJECTID,Name,Stacked,AnnoLine1,AnnoLine2,AnnoLine3,AnnoAngle,Borough,long,lat
0,1,Wakefield,1,Wakefield,NaN,NaN,0,Bronx,-73.8472005205491,40.89470517661004
1,2,Co-op City,2,Co-op,City,NaN,0,Bronx,-73.82993910812405,40.87429419303015
2,3,Eastchester,1,Eastchester,NaN,NaN,0,Bronx,-73.82780644716419,40.88755567735082
3,4,Fieldston,1,Fieldston,NaN,NaN,0,Bronx,-73.90564259591689,40.895437426903875
4,5,Riverdale,1,Riverdale,NaN,NaN,0,Bronx,-73.91258546108577,40.89083449389134


### Drop unnecessary columns.

In [7]:
df2.drop(['OBJECTID','Stacked','AnnoLine1','AnnoLine2','AnnoLine3','AnnoAngle'], axis=1, inplace=True)

df2.head()

,Name,Borough,long,lat
0,Wakefield,Bronx,-73.8472005205491,40.89470517661004
1,Co-op City,Bronx,-73.82993910812405,40.87429419303015
2,Eastchester,Bronx,-73.82780644716419,40.88755567735082
3,Fieldston,Bronx,-73.90564259591689,40.895437426903875
4,Riverdale,Bronx,-73.91258546108577,40.89083449389134


#### Check the dataframe.

In [8]:
df2.dtypes

Name       object
Borough    object
long       object
lat        object
dtype: object

#### Change lat and long columns to float.

In [9]:
df2['long'] = df2.long.astype(float)
df2['lat'] = df2.lat.astype('str').astype(float)
df2 = df2[['Name','Borough','lat','long']]
df2.head()

,Name,Borough,lat,long
0,Wakefield,Bronx,40.894705,-73.847201
1,Co-op City,Bronx,40.874294,-73.829939
2,Eastchester,Bronx,40.887556,-73.827806
3,Fieldston,Bronx,40.895437,-73.905643
4,Riverdale,Bronx,40.890834,-73.912585


#### Give columns descriptive names.

In [10]:
df2.rename(columns={'Name': 'Neighborhood', 'lat': 'Latitude', 'long': 'Longitude'}, inplace = True)
#df2.to_csv('neighborhood geo.csv')
df2.head()


,Neighborhood,Borough,Latitude,Longitude
0,Wakefield,Bronx,40.894705,-73.847201
1,Co-op City,Bronx,40.874294,-73.829939
2,Eastchester,Bronx,40.887556,-73.827806
3,Fieldston,Bronx,40.895437,-73.905643
4,Riverdale,Bronx,40.890834,-73.912585


#### Print data for a quality check.

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df2['Borough'].unique()),
        df2.shape[0]
    )
)

The dataframe has 5 boroughs and 299 neighborhoods.


## Get Population Data from Wikipedia
#### New York City's website lacks population data for each neighborhood. Instead, the NYC Planning Department groups some neighborhoods together into what they call Neighborhood Tabulation Areas (NTA). Because of this, I will try to get the data from Wikipedia.

#### Create a function and use BeautifulSoup to scrape population data from Wikipedia.

In [12]:
from bs4 import BeautifulSoup

def get_neighborhood_population(read_from_csv=False):
    if not read_from_csv:
        WIKI_LINK = "https://en.wikipedia.org/wiki/Neighborhoods_in_New_York_City"
        ROOT_WIKI_LINK = "https://en.wikipedia.org"
        page = requests.get(WIKI_LINK)
        soup = BeautifulSoup(page.text, 'html.parser')
        population_list = []
        for table_row in soup.select("table.wikitable tr"):
            cells = table_row.findAll('td')
            if len(cells) > 0:
                borough = cells[0].text.strip().replace(
                    '\xa0', ' ').split(' ')[0]
                population = int(cells[3].text.strip().replace(',', ''))
                for item in cells[4].findAll('a'):
                    neighborhood = item.text
                    neighbourhood_page = requests.get(
                        ROOT_WIKI_LINK+item['href'])
                    soup = BeautifulSoup(
                        neighbourhood_page.text, 'html.parser')
                    table = soup.select("table.infobox tr")
                    should_record = False
                    for row in table:
                        head = row.find('th')
                        body = row.find('td')
                        if head and 'population' in head.text.lower():
                            should_record = True
                            continue
                        if should_record:
                            try:
                                population_list.append(
                                    [borough, neighborhood, int(body.text.replace(',', ''))])
                            except:
                                pass
                            should_record = False
        df = pd.DataFrame(population_list, columns=[
                          "Borough", "Neighborhood", "Population"])
        df.to_csv('population.csv')
    else:
        df = pd.read_csv('population.csv')
    df = df.sort_values(by=['Borough'])
    df = df.drop_duplicates(subset='Neighborhood', keep='last')
    return df


In [13]:
nyc_neighborhood_pop_df = get_neighborhood_population()

In [14]:
print(nyc_neighborhood_pop_df.shape)
nyc_neighborhood_pop_df.head()

(183, 3)


,Borough,Neighborhood,Population
0,Bronx,Melrose,24913
25,Bronx,Bruckner,38557
26,Bronx,Castle Hill,38557
27,Bronx,Clason Point,9136
28,Bronx,Harding Park,9136


#### As you can see by looking at the shape above, some neighborhoods are missing data (started with 299 rows and ended up with 183). I will fix the missing data below when I choose which borough to focus on.

In [15]:
nyc_neighborhood_pop_df.dtypes

Borough         object
Neighborhood    object
Population       int64
dtype: object

## Combine the neighborhood geospatial dataframe with the population dataframe.
#### By left merging the data from df2 with the population dataframe, we will have all neighborhoods but will have missing population values.

In [16]:
df2.set_index('Neighborhood')
nyc_neighborhood_pop_df.set_index('Neighborhood')
nyc_df = pd.merge(df2, nyc_neighborhood_pop_df, how="left", on=["Borough", "Neighborhood"])
nyc_df.to_csv('neighborhood population.csv')
nyc_df.head()

,Neighborhood,Borough,Latitude,Longitude,Population
0,Wakefield,Bronx,40.894705,-73.847201,29158.0
1,Co-op City,Bronx,40.874294,-73.829939,43752.0
2,Eastchester,Bronx,40.887556,-73.827806,NaN
3,Fieldston,Bronx,40.895437,-73.905643,3292.0
4,Riverdale,Bronx,40.890834,-73.912585,48049.0


In [17]:
print(nyc_df.shape)
nyc_df.dtypes

(299, 5)


Neighborhood     object
Borough          object
Latitude        float64
Longitude       float64
Population      float64
dtype: object

## Create New York City Map of the Neighborhoods

In [18]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City is 40.7127281, -74.0060152.


In [19]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, pop in zip(nyc_df['Latitude'], nyc_df['Longitude'], nyc_df['Borough'], nyc_df['Neighborhood'], nyc_df['Population']):
    label = '{}, {}, Population: {}'.format(neighborhood, borough, pop)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  

map_newyork

## Get hospital location data for the Bronx borough
#### According to NYC.gov data on April 21, the Bronx has the highest rate of cases compared to the other 4 boroughs. Thus, I'll focus on finding locations in the Bronx.

In [97]:
bronx_data = nyc_df[nyc_df['Borough'] == 'Bronx'].reset_index(drop=True)
#bronx_data.drop(['Population'], axis=1, inplace=True)
bronx_data.to_csv('bronx data.csv')
print(bronx_data.shape)
bronx_data.head()


(49, 5)


,Neighborhood,Borough,Latitude,Longitude,Population
0,Wakefield,Bronx,40.894705,-73.847201,29158.0
1,Co-op City,Bronx,40.874294,-73.829939,43752.0
2,Eastchester,Bronx,40.887556,-73.827806,NaN
3,Fieldston,Bronx,40.895437,-73.905643,3292.0
4,Riverdale,Bronx,40.890834,-73.912585,48049.0


#### Missing Wikipedia population data was corrected by going to city-data.com, the site that Wikipedia uses for population data, and updating 'bronx data.csv' and then imported that back into the notebook below.

In [103]:
bronx_data_updated = pd.read_csv('bronx data updated.csv')
bronx_data_updated.drop(['Unnamed: 0'], axis=1, inplace=True)

bronx_data_updated

,Neighborhood,Borough,Latitude,Longitude,Population
0,Wakefield,Bronx,40.894705,-73.847201,29158.0
1,Co-op City,Bronx,40.874294,-73.829939,43752.0
2,Eastchester,Bronx,40.887556,-73.827806,10739.0
3,Fieldston,Bronx,40.895437,-73.905643,3292.0
4,Riverdale,Bronx,40.890834,-73.912585,48049.0
5,Kingsbridge,Bronx,40.881687,-73.902818,10669.0
6,Woodlawn,Bronx,40.898273,-73.867315,42483.0
7,Norwood,Bronx,40.877224,-73.879391,40494.0
8,Williamsbridge,Bronx,40.881039,-73.857446,61321.0
9,Baychester,Bronx,40.866858,-73.835798,63345.0


#### Formatting data to work with k-means clustering.

In [104]:
bronx_data_updated['Population'] = bronx_data_updated['Population'].astype(int)

In [105]:
bronx_data = bronx_data_updated
bronx_data

,Neighborhood,Borough,Latitude,Longitude,Population
0,Wakefield,Bronx,40.894705,-73.847201,29158
1,Co-op City,Bronx,40.874294,-73.829939,43752
2,Eastchester,Bronx,40.887556,-73.827806,10739
3,Fieldston,Bronx,40.895437,-73.905643,3292
4,Riverdale,Bronx,40.890834,-73.912585,48049
5,Kingsbridge,Bronx,40.881687,-73.902818,10669
6,Woodlawn,Bronx,40.898273,-73.867315,42483
7,Norwood,Bronx,40.877224,-73.879391,40494
8,Williamsbridge,Bronx,40.881039,-73.857446,61321
9,Baychester,Bronx,40.866858,-73.835798,63345


#### Let's get the geographical coordinates of the Bronx.

In [143]:
address = 'The Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the Bronx are 40.8466508, -73.8785937.


#### Create a map of the Bronx and the neighborhoods

In [146]:
# create map of Manhattan using latitude and longitude values
map_bronx = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, label, pop in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood'], bronx_data['Population']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=0.0007*pop,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

## Use Foursquare API to gather hospital data

#### Define Foursquare credentials and version.

In [139]:
CLIENT_ID = 'UVSCOAPXL3LJQXQS2AUCATQR3ZZXN05R1FTL3HI0HHFR4IZK' # your Foursquare ID
CLIENT_SECRET = 'JDW330LUZTAGQFRBDBF4X4TOOQ4SVUO1FUPAFOBNHEZFJJHK' # your Foursquare Secret
Foursquare_ER_Key = '4bf58dd8d48988d194941735'
VERSION = '20200420' # Foursquare API version
limit = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UVSCOAPXL3LJQXQS2AUCATQR3ZZXN05R1FTL3HI0HHFR4IZK
CLIENT_SECRET:JDW330LUZTAGQFRBDBF4X4TOOQ4SVUO1FUPAFOBNHEZFJJHK


#### Create GET request URL and function to get hospital data.

In [140]:
def getHospitals(names, latitudes, longitudes, radius=500):

    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit,
            Foursquare_ER_Key)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    hospitals = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    hospitals.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Hospital', 
                  'Hospital Latitude', 
                  'Hospital Longitude', 
                  'Hospital Category']
    
    return hospitals 


In [141]:
venues = getHospitals(names=bronx_data['Neighborhood'], 
                      latitudes=bronx_data['Latitude'],
                      longitudes=bronx_data['Longitude']
                          )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope


In [111]:
print(venues.shape)
venues.to_csv('bronx hospitals.csv')
venues

(26, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Hospital,Hospital Latitude,Hospital Longitude,Hospital Category
0,Woodlawn,40.898273,-73.867315,Station 27,40.896225,-73.869066,Emergency Room
1,Norwood,40.877224,-73.879391,North Central Pediatric Emergency Room,40.879268,-73.881167,Emergency Room
2,Norwood,40.877224,-73.879391,Montefiore Medical Center,40.879938,-73.880868,Hospital
3,Norwood,40.877224,-73.879391,Montefiore Emergency Room,40.880650,-73.878778,Emergency Room
4,Norwood,40.877224,-73.879391,Montefiore Medical Center Pediatric ER,40.880899,-73.879398,Emergency Room
5,Norwood,40.877224,-73.879391,Montefiore Medical Center Moses Division Emerg...,40.880992,-73.878932,Emergency Room
6,Bedford Park,40.870185,-73.885512,Montefiore Ems Als 27X2,40.868792,-73.887265,Emergency Room
7,University Heights,40.855727,-73.910416,morris heights urgent care,40.854361,-73.909702,Emergency Room
8,Morris Heights,40.847898,-73.919672,NYC EMS 17S CSL,40.849796,-73.915810,Emergency Room
9,Morris Heights,40.847898,-73.919672,NYC EMS 19C CSL,40.850372,-73.915544,Emergency Room


### Create a map of Emergency Room locations in relationship with the neighborhoods.

In [147]:
map_Hospitals = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, neighborhood, pop in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood'], bronx_data['Population']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=0.0007*pop,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Hospitals)

#Hospital markers
for lat, lng, hospital in zip(venues['Hospital Latitude'], venues['Hospital Longitude'], venues['Hospital']):
    label = '{}'.format(hospital)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(map_Hospitals)

map_Hospitals

## Analyze Neighborhoods

In [114]:
bronx_onehot = pd.get_dummies(venues[['Hospital Category']], prefix="", prefix_sep="")
bronx_onehot['Neighborhood'] = venues['Neighborhood'] 

fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]

bronx_onehot

,Neighborhood,Emergency Room,Hospital
0,Woodlawn,1,0
1,Norwood,1,0
2,Norwood,0,1
3,Norwood,1,0
4,Norwood,1,0
5,Norwood,1,0
6,Bedford Park,1,0
7,University Heights,1,0
8,Morris Heights,1,0
9,Morris Heights,1,0


In [115]:
bronx_onehot.shape

(26, 3)

### Group rows by neighborhood with a hospital and find the mean.

In [116]:
bronx_grouped = bronx_onehot.groupby('Neighborhood').mean().reset_index()
bronx_grouped

,Neighborhood,Emergency Room,Hospital
0,Bedford Park,1.000000,0.000000
1,Castle Hill,1.000000,0.000000
2,Morris Heights,1.000000,0.000000
3,Morris Park,0.666667,0.333333
4,Mount Eden,1.000000,0.000000
5,Mount Hope,1.000000,0.000000
6,Norwood,0.800000,0.200000
7,Pelham Bay,1.000000,0.000000
8,Pelham Gardens,1.000000,0.000000
9,University Heights,1.000000,0.000000


In [117]:
bronx_grouped.shape

(11, 3)

### Add in neighborhoods without a hospital.

In [118]:
bronx = pd.merge(bronx_grouped, bronx_data, how='outer', on=["Neighborhood"])
bronx.fillna(0, inplace=True)
bronx.drop(['Borough','Latitude','Longitude','Population'], axis=1, inplace=True)
bronx

,Neighborhood,Emergency Room,Hospital
0,Bedford Park,1.000000,0.000000
1,Castle Hill,1.000000,0.000000
2,Morris Heights,1.000000,0.000000
3,Morris Park,0.666667,0.333333
4,Mount Eden,1.000000,0.000000
5,Mount Hope,1.000000,0.000000
6,Norwood,0.800000,0.200000
7,Pelham Bay,1.000000,0.000000
8,Pelham Gardens,1.000000,0.000000
9,University Heights,1.000000,0.000000


## Cluster the neighborhoods

In [119]:
# set number of clusters
kclusters = 5

bronx_grouped_clustering = bronx.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 1, 1, 2, 1, 1, 1], dtype=int32)

In [120]:
# add clustering labels
bronx.insert(0, 'Cluster Labels', kmeans.labels_)

bronx_merged = bronx_data

# merge bronx_grouped with bronx_data to add latitude/longitude for each neighborhood
bronx_merged = bronx_merged.join(bronx.set_index('Neighborhood'), on='Neighborhood')

bronx_merged # check the last columns!

,Neighborhood,Borough,Latitude,Longitude,Population,Cluster Labels,Emergency Room,Hospital
0,Wakefield,Bronx,40.894705,-73.847201,29158,0,0.000000,0.000000
1,Co-op City,Bronx,40.874294,-73.829939,43752,0,0.000000,0.000000
2,Eastchester,Bronx,40.887556,-73.827806,10739,0,0.000000,0.000000
3,Fieldston,Bronx,40.895437,-73.905643,3292,0,0.000000,0.000000
4,Riverdale,Bronx,40.890834,-73.912585,48049,0,0.000000,0.000000
5,Kingsbridge,Bronx,40.881687,-73.902818,10669,0,0.000000,0.000000
6,Woodlawn,Bronx,40.898273,-73.867315,42483,1,1.000000,0.000000
7,Norwood,Bronx,40.877224,-73.879391,40494,2,0.800000,0.200000
8,Williamsbridge,Bronx,40.881039,-73.857446,61321,0,0.000000,0.000000
9,Baychester,Bronx,40.866858,-73.835798,63345,0,0.000000,0.000000


In [121]:
#bronx_merged.fillna(0, inplace=True)
#bronx_merged['Cluster Labels'] = bronx_merged['Cluster Labels'].astype(int)
#bronx_merged

,Neighborhood,Borough,Latitude,Longitude,Population,Cluster Labels,Emergency Room,Hospital
0,Wakefield,Bronx,40.894705,-73.847201,29158,0,0.000000,0.000000
1,Co-op City,Bronx,40.874294,-73.829939,43752,0,0.000000,0.000000
2,Eastchester,Bronx,40.887556,-73.827806,10739,0,0.000000,0.000000
3,Fieldston,Bronx,40.895437,-73.905643,3292,0,0.000000,0.000000
4,Riverdale,Bronx,40.890834,-73.912585,48049,0,0.000000,0.000000
5,Kingsbridge,Bronx,40.881687,-73.902818,10669,0,0.000000,0.000000
6,Woodlawn,Bronx,40.898273,-73.867315,42483,1,1.000000,0.000000
7,Norwood,Bronx,40.877224,-73.879391,40494,2,0.800000,0.200000
8,Williamsbridge,Bronx,40.881039,-73.857446,61321,0,0.000000,0.000000
9,Baychester,Bronx,40.866858,-73.835798,63345,0,0.000000,0.000000


## Create a map that shows population by size of the markers as well as Hospital locations.

In [138]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, pop, cluster in zip(bronx_merged['Latitude'], bronx_merged['Longitude'], bronx_merged['Neighborhood'], bronx_merged['Population'], bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=0.0007 * pop,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

#Hospital markers
for lat, lng, hospital in zip(venues['Hospital Latitude'], venues['Hospital Longitude'], venues['Hospital']):
    label = '{}'.format(hospital)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
        [lat, lng],
        popup=label,
        icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(map_clusters)
       
map_clusters

### Examine Clusters

In [123]:
bronx_merged[(bronx_merged['Cluster Labels'] == 0)]

,Neighborhood,Borough,Latitude,Longitude,Population,Cluster Labels,Emergency Room,Hospital
0,Wakefield,Bronx,40.894705,-73.847201,29158,0,0.0,0.0
1,Co-op City,Bronx,40.874294,-73.829939,43752,0,0.0,0.0
2,Eastchester,Bronx,40.887556,-73.827806,10739,0,0.0,0.0
3,Fieldston,Bronx,40.895437,-73.905643,3292,0,0.0,0.0
4,Riverdale,Bronx,40.890834,-73.912585,48049,0,0.0,0.0
5,Kingsbridge,Bronx,40.881687,-73.902818,10669,0,0.0,0.0
8,Williamsbridge,Bronx,40.881039,-73.857446,61321,0,0.0,0.0
9,Baychester,Bronx,40.866858,-73.835798,63345,0,0.0,0.0
10,Pelham Parkway,Bronx,40.857413,-73.854756,30073,0,0.0,0.0
11,City Island,Bronx,40.847247,-73.786488,4225,0,0.0,0.0


In [124]:
bronx_merged[(bronx_merged['Cluster Labels'] == 1)]

,Neighborhood,Borough,Latitude,Longitude,Population,Cluster Labels,Emergency Room,Hospital
6,Woodlawn,Bronx,40.898273,-73.867315,42483,1,1.0,0.0
12,Bedford Park,Bronx,40.870185,-73.885512,37344,1,1.0,0.0
13,University Heights,Bronx,40.855727,-73.910416,25702,1,1.0,0.0
14,Morris Heights,Bronx,40.847898,-73.919672,36779,1,1.0,0.0
36,Pelham Bay,Bronx,40.850641,-73.832074,11931,1,1.0,0.0
39,Castle Hill,Bronx,40.819014,-73.848027,38557,1,1.0,0.0
41,Pelham Gardens,Bronx,40.862966,-73.841612,15388,1,1.0,0.0
47,Mount Eden,Bronx,40.843826,-73.916556,20236,1,1.0,0.0
48,Mount Hope,Bronx,40.848842,-73.908299,24739,1,1.0,0.0


In [125]:
bronx_merged[(bronx_merged['Cluster Labels'] == 2)]

,Neighborhood,Borough,Latitude,Longitude,Population,Cluster Labels,Emergency Room,Hospital
7,Norwood,Bronx,40.877224,-73.879391,40494,2,0.8,0.2


In [126]:
bronx_merged[(bronx_merged['Cluster Labels'] == 3)]

,Neighborhood,Borough,Latitude,Longitude,Population,Cluster Labels,Emergency Room,Hospital
32,Morris Park,Bronx,40.847549,-73.850402,11019,3,0.666667,0.333333


## Summary

#### From looking at the map and the dataframes above, it appears that the south Bronx region, particularly the Mott Haven-Melrose-Longwood area, could use a temporary medical facility to help with COVID-19 treatment.